In [ ]:
import pandas as pd
import requests
import os 

In [4]:
df10 = pd.read_csv('books10.csv')

In [5]:
system_prompt = """
    You are a helpful literary assistant, helping me with my master thesis on gender in literature. 
    Your job is to search the internet and assess whether a book features a female main character.
    The character does not need to be the sole protagonist but must play a central role in the plot, 
    with more significance than a secondary or supporting character.
    Reason well internally, but respond only with 'Yes' or 'No'.
    """

user_prompt_template = "Is there a female main character in the book {title} by {author}?"

In [6]:
# Setting my API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Saying hi it's me, and this is what I'll be sending
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# This is where I want to send it to
url = "https://api.perplexity.ai/chat/completions"

# I'll store the responses I get in this (as of now, empty) list
responses = []

# Setting up what I want to send to Perplexity
for index, row in df10.iterrows():
    # Extract title and author from each row 
    title = row["title"]
    author = row["author"]

    # Format user prompt with current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)

    # Payload
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        # "max_tokens": 123,  # Limits the length of the response
        # "temperature": 0.2,  # Controls randomness in the response
        # "top_p": 0.9,
        # "search_domain_filter": ["<any>"],
        # "return_images": False,
        # "return_related_questions": False,
        # "search_recency_filter": "<string>",
        # "top_k": 0,
        # "stream": False,
        # "presence_penalty": 0,
        # "frequency_penalty": 1,
        # "response_format": {},
        # "web_search_options": {"search_context_size": "high"}
    }

    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response (this might vary depending on the response format)
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        responses.append(answer)  # Store the answer
    else:
        # Handle errors by appending an error message
        responses.append(f"Error: {response.status_code}")

# Add the responses to your original dataframe
df10['female_main_character'] = responses

# Print the updated dataframe to check the results
print(df10)

                                               title               author  \
0   The Mysterious Disappearance of Leon I Mean Noel        Raskin, Ellen   
1          The Summer I Turned Pretty The Collection         Han, Jenny *   
2                        Complete Me Stark Trilogy 3         Kenner, J. *   
3          One More Chance Rosemary Beach 8 Chance 2       Glines, Abbi *   
4                  Not a Penny More Not a Penny Less    Archer, Jeffrey *   
5                  Sign Off Caught Dead in Wyoming 1   McLinn, Patricia *   
6  Max Havelaar or the Coffee Auctions of the Dut...            Multatuli   
7                                         First Love       Turgenev, Ivan   
8                        Ship Breaker Ship Breaker 1  Bacigalupi, Paolo *   
9                      In the Country of Last Things         Auster, Paul   

  female_main_character  
0                   Yes  
1                   Yes  
2                  Yes.  
3                   Yes  
4                    N

Try in batch? 